In [2]:
import pandas as pd
import fba_utils
from fba_utils import get_flux_bounds, convert_transcriptomics_to_enzyme_activity
import cobra
import numpy as np
import os 

In [3]:
os.getcwd()

'/home/connah/Documents/Repos/syn_bmca/notebooks/notebooks_for_generating_data'

## 1) collect input

In [4]:
# get the cobra model for S. elongatus
syn_model = cobra.io.read_sbml_model('../../models/syn_elong.xml')
[m.id for m in syn_model.metabolites if m.id.startswith('sucr') ]

Restricted license - for non-production use only - expires 2025-11-24


['sucr_c', 'sucr_e']

In [5]:
# get the metabolomics data, this is only for sucrose activity/abundance
metab_df = pd.read_csv('../../data/circadian_experiments/processed_data/sucrose_metabolomics.csv',index_col=0)
display(metab_df)

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
sucr_e,0.644,0.654,0.681,0.881,0.811,1.031,0.829,1.006,0.964,1.811,...,0.788,2.17,2.231,2.27,2.404,3.216,3.098,4.431,3.103,4.188


In [6]:
display(metab_df.columns)

Index(['L_T0_A', 'L_T0_B', 'L_T0_C', 'L_T2_A', 'L_T2_B', 'L_T2_C', 'L_T4_A',
       'L_T4_B', 'L_T4_C', 'L_T8_A', 'L_T8_B', 'L_T8_C', 'L_T12_A', 'L_T12_B',
       'L_T12_C', 'L_T16_A', 'L_T16_B', 'L_T16_C', 'D_T0_A', 'D_T0_B',
       'D_T0_C', 'D_T2_A', 'D_T2_B', 'D_T2_C', 'D_T4_A', 'D_T4_B', 'D_T4_C',
       'D_T8_A', 'D_T8_B', 'D_T8_C'],
      dtype='object')

In [7]:
# get the transcriptomics data 
transcript_df = pd.read_csv('../../data/circadian_experiments/raw_data/Se_Sucrose_circadian_data/Counts_ESeq2median_norm.csv',index_col=0)
display(transcript_df)

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
SYNPCC7942_RS00005,607.927550,582.795478,551.827288,975.630070,884.915418,822.426446,975.113687,961.357516,1005.261196,1031.011880,...,903.383312,1319.278749,1446.428293,962.039815,1201.548782,1200.140721,1160.674229,951.044468,944.250567,923.493988
SYNPCC7942_RS00010,302.461483,289.783347,286.620741,262.244540,260.796220,223.922884,237.446515,220.505547,248.107019,157.592774,...,439.786605,399.962336,431.345580,335.576388,316.650821,267.897315,263.509184,184.590910,201.161646,184.045382
SYNPCC7942_RS00015,2057.138694,1768.566334,1912.452184,2982.152470,2965.312745,2368.216681,2620.354206,2786.070083,2380.544066,2374.989692,...,1315.858328,2560.100067,2721.523710,2326.283777,2744.045848,2624.019860,2781.387903,3281.170296,2854.736581,3055.806756
SYNPCC7942_RS00020,4618.046018,4923.088121,3877.616229,3263.487609,3846.246542,3090.548566,2103.248462,2291.390973,2263.976545,1843.391532,...,1503.537962,1682.570764,1864.859335,2109.337298,1793.309598,2115.701876,2062.245784,3516.590587,3250.464409,2721.475796
SYNPCC7942_RS00025,276.421753,274.446624,289.915232,429.036087,653.981361,803.852198,639.522613,803.853554,928.262466,910.042779,...,654.077530,600.369903,677.582778,723.967462,573.733665,708.504988,825.779615,272.873518,286.902675,326.707779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HTX97_RS00025,17.025978,17.758311,17.296079,25.119209,60.719731,63.977967,28.493582,44.334449,28.874524,28.855015,...,31.513371,54.579082,35.299738,33.313879,29.000199,37.289736,40.539874,36.115613,24.183367,21.780519
HTX97_RS00030,45.068764,43.588581,36.239404,62.295638,73.660001,65.009870,61.208435,73.501849,66.304462,39.953098,...,46.219611,67.371054,68.016569,56.877354,35.270512,53.971987,60.809812,41.466074,36.275051,35.937856
HTX97_RS00035,3.004584,0.000000,0.000000,4.019073,1.990811,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.400594,0.852798,2.582908,0.812534,0.783789,1.962618,0.881302,1.337615,1.099244,3.267078
HTX97_RS00040,7.010697,12.107939,16.472456,17.081062,4.977027,9.287124,5.276589,16.333744,7.485988,9.988274,...,18.207726,20.467156,13.775508,6.500269,7.837892,17.663559,8.813016,10.700922,5.496220,5.445130


In [8]:
# get the proteomic data 
protein_df = pd.read_csv('../../data/circadian_experiments/raw_data/processed_global_proteomics_SYN_BMCA.csv',index_col=0)
display(protein_df)

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
Protein,,,,,,,,,,,,,,,,,,,,,
6PGD_SYNE7,24.855182,24.774480,24.855081,24.656410,24.625907,24.666508,24.590587,24.520038,24.530295,24.495133,...,24.996710,24.970787,24.888715,24.964703,24.907633,24.856183,24.866019,24.638474,24.659501,24.695780
A8DZ36_SYNE7,19.779120,19.759664,19.782483,19.914904,19.965987,19.997244,19.927738,19.926499,19.901003,19.822698,...,20.070924,20.017955,20.173828,20.071358,20.117915,20.026412,20.149414,20.033935,20.024065,19.993526
AAR_SYNE7,23.218211,23.141061,23.224484,23.252884,23.258088,23.215281,23.336923,23.310040,23.339360,23.391776,...,22.914969,23.026786,23.026106,22.996076,23.198511,23.219795,23.256874,23.457288,23.478206,23.502473
ACCA_SYNE7,22.816063,22.800831,22.871235,22.774508,22.746694,22.755746,22.809989,22.814432,22.755420,22.700772,...,22.739096,22.753632,22.714166,22.755254,22.839679,22.780423,22.737947,22.740097,22.654780,22.771759
ACCD_SYNE7,23.529186,23.534297,23.573745,23.451529,23.430393,23.450602,23.418942,23.351758,23.424391,23.456726,...,23.492077,23.450201,23.419010,23.468640,23.449409,23.493616,23.507865,23.504392,23.528526,23.523663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YCF3_SYNE7,21.371221,21.351003,21.348168,21.308436,21.283660,21.347702,21.392678,21.378683,21.375391,21.620661,...,21.303695,21.366746,21.108122,21.202759,21.261464,21.188096,21.348947,21.582797,21.458388,21.467279
YCF4_SYNE7,22.498840,22.454164,22.564172,22.490923,22.331240,22.491240,22.453242,22.399541,22.316991,22.305373,...,22.279263,22.265819,22.168122,22.195117,22.142730,22.147244,22.126451,22.132577,22.140331,22.220989
YCF48_SYNE7,24.388791,24.281924,24.420212,24.234235,24.113444,24.149871,24.146244,24.088103,24.069248,24.034192,...,24.198663,24.088847,24.108195,24.070107,24.069673,24.093871,24.055132,24.045308,24.059602,24.133706


## 2) Calculate additional data

In [9]:
# calculate enzyme activity
# Run enzyme activity converter for all strains in transcriptomics_df
all_enzyme_activity_df = convert_transcriptomics_to_enzyme_activity(transcript_df, syn_model)
display(all_enzyme_activity_df)


,Reaction_ID,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
EX_gln__L_e: gln__L_e -->,EX_gln__L_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EX_hco3_e: hco3_e <=>,EX_hco3_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EX_mn2_e: mn2_e <=>,EX_mn2_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EX_arg__L_e: arg__L_e -->,EX_arg__L_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADPT: ade_c + prpp_c <=> amp_c + ppi_c,ADPT,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZDS: h_c + nadph_c + o2_c + zcarote_c --> 2.0 h2o_c + nadp_c + norsp_c,ZDS,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
RDXRr: nadh_c + 2.0 rdxo_c <=> h_c + nad_c + 2.0 rdxr_c,RDXRr,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
MPTSS: atp_c + h_c + moadcoo_c --> moadamp_c + ppi_c,MPTSS,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
NOR: h_c + nadph_c + norsp_c + o2_c --> 2.0 h2o_c + lyc_c + nadp_c,NOR,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


In [10]:
# check the enzyme conversion
all_enzyme_activity_df['L_T0_A'].dropna().min()

11.01680898

In [11]:
### Cleanup enzyme activity df
clean_enzyme_activity_df = all_enzyme_activity_df.copy()

# Replace current index with Reaction_ID column
clean_enzyme_activity_df = clean_enzyme_activity_df.set_index('Reaction_ID')

# Drop rows with all 0s: 
clean_enzyme_activity_df = clean_enzyme_activity_df.loc[~(clean_enzyme_activity_df==0).all(axis=1)]

# Drop rows with all infs:
clean_enzyme_activity_df.replace([np.inf, -np.inf], np.nan, inplace=True)
clean_enzyme_activity_df.dropna(how='all', inplace=True)

clean_enzyme_activity_df

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
DTMPK,178.272000,156.596015,187.786003,160.762936,123.430272,114.541199,132.970048,140.003522,116.567522,169.800665,...,207.988251,185.909998,149.808645,138.130717,172.433615,116.775753,138.364354,145.800066,195.665426,149.196552
ACOATA,404.617348,452.836930,367.335777,583.770413,636.064063,599.535465,566.705682,541.346951,544.338243,432.825224,...,293.424502,569.669169,588.902948,576.898876,582.355346,525.000234,539.356590,583.200265,533.133324,503.129979
3HAD160,241.368269,171.125542,172.137169,183.872609,116.462434,99.062659,165.684901,131.836650,104.803827,178.679131,...,280.118856,264.367429,224.712967,82.878430,169.298459,158.972033,155.109085,268.860673,193.466938,235.229601
RNDR3,2704.125840,2629.037221,2801.141205,5320.248426,6440.273066,6515.433605,4435.500897,4978.291895,4880.863935,5450.268400,...,1703.122647,5425.501875,4777.518212,6622.961601,6265.610544,7346.078037,7799.519310,10922.966420,5126.873860,4654.496820
G5SD,1862.842245,1853.321909,1853.974964,1931.164774,1893.261108,1781.064048,1973.444367,2080.218994,1932.454235,1725.751855,...,2043.467058,2109.822641,2228.188346,1980.956985,2197.744805,2174.580408,2157.426358,1872.661400,1917.081478,1769.667134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MDDCP5pp,862.315684,912.938624,885.394530,917.353506,1001.377852,1238.283232,1020.492355,1135.195222,1150.703241,1019.913798,...,1110.671266,905.671643,903.156714,1032.730241,814.356937,833.131212,842.524346,741.038868,718.905554,824.392628
MDDEP2pp,2088.186065,2206.066541,2120.005135,1920.112322,2121.208949,2432.194648,2667.843509,2781.403299,2508.875283,1981.007757,...,2303.277298,2323.022180,2327.199807,2145.901311,1570.713476,1817.383987,1816.362633,1322.901518,1172.893312,1275.249363
MDDEP3pp,2088.186065,2206.066541,2120.005135,1920.112322,2121.208949,2432.194648,2667.843509,2781.403299,2508.875283,1981.007757,...,2303.277298,2323.022180,2327.199807,2145.901311,1570.713476,1817.383987,1816.362633,1322.901518,1172.893312,1275.249363


## 3) Process global proteomics (protein abundance) data

In [14]:
# no notes given to process so preserve the data

## 4) Normalize dataframes with respect to reference condition

In [12]:
# define reference condition
ref_cond = [c for c in metab_df.columns if '0' in c and 'A' in c][0]
display(ref_cond)

'L_T0_A'

In [13]:
# normalize metabolomics data
norm_metab_df = metab_df.div(metab_df[ref_cond],axis=0)
display(norm_metab_df)
display(norm_metab_df[ref_cond])

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
sucr_e,1.0,1.015528,1.057453,1.368012,1.259317,1.600932,1.287267,1.562112,1.496894,2.812112,...,1.223602,3.369565,3.464286,3.524845,3.732919,4.993789,4.810559,6.880435,4.818323,6.503106


sucr_e    1.0
Name: L_T0_A, dtype: float64

In [14]:
# normalize transcriptomics data
norm_enzyme_activity_df = clean_enzyme_activity_df.div(clean_enzyme_activity_df[ref_cond],axis=0)
display(norm_enzyme_activity_df)
display(norm_enzyme_activity_df[ref_cond])



,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
DTMPK,1.0,0.878411,1.053368,0.901785,0.692370,0.642508,0.745883,0.785337,0.653875,0.952481,...,1.166691,1.042845,0.840337,0.774831,0.967250,0.655043,0.776142,0.817852,1.097567,0.836904
ACOATA,1.0,1.119173,0.907860,1.442772,1.572014,1.481734,1.400597,1.337923,1.345316,1.069715,...,0.725190,1.407921,1.455456,1.425789,1.439274,1.297523,1.333004,1.441362,1.317623,1.243471
3HAD160,1.0,0.708981,0.713172,0.761793,0.482509,0.410421,0.686440,0.546205,0.434207,0.740276,...,1.160545,1.095287,0.930996,0.343369,0.701411,0.658629,0.642624,1.113902,0.801543,0.974567
RNDR3,1.0,0.972232,1.035877,1.967456,2.381647,2.409442,1.640272,1.840999,1.804969,2.015538,...,0.629824,2.006379,1.766751,2.449206,2.317056,2.716618,2.884303,4.039371,1.895945,1.721257
G5SD,1.0,0.994889,0.995240,1.036676,1.016329,0.956100,1.059373,1.116691,1.037369,0.926408,...,1.096962,1.132583,1.196123,1.063406,1.179780,1.167345,1.158137,1.005271,1.029116,0.949982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MDDCP5pp,1.0,1.058706,1.026764,1.063826,1.161266,1.435998,1.183432,1.316450,1.334434,1.182762,...,1.288010,1.050279,1.047362,1.197624,0.944384,0.966156,0.977049,0.859359,0.833692,0.956022
MDDEP2pp,1.0,1.056451,1.015238,0.919512,1.015814,1.164740,1.277589,1.331971,1.201462,0.948674,...,1.103004,1.112459,1.114460,1.027639,0.752190,0.870317,0.869828,0.633517,0.561680,0.610697
MDDEP3pp,1.0,1.056451,1.015238,0.919512,1.015814,1.164740,1.277589,1.331971,1.201462,0.948674,...,1.103004,1.112459,1.114460,1.027639,0.752190,0.870317,0.869828,0.633517,0.561680,0.610697


Reaction_ID
DTMPK       1.0
ACOATA      1.0
3HAD160     1.0
RNDR3       1.0
G5SD        1.0
           ... 
MDDCP5pp    1.0
MDDEP2pp    1.0
MDDEP3pp    1.0
MDDEP4pp    1.0
MNHNAtpp    1.0
Name: L_T0_A, Length: 682, dtype: float64

In [15]:
norm_protein_df = protein_df.div(protein_df[ref_cond],axis=0)
display(norm_protein_df)
display(norm_protein_df[ref_cond])

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
Protein,,,,,,,,,,,,,,,,,,,,,
6PGD_SYNE7,1.0,0.996753,0.999996,0.992003,0.990776,0.992409,0.989355,0.986516,0.986929,0.985514,...,1.005694,1.004651,1.001349,1.004406,1.002110,1.000040,1.000436,0.991281,0.992127,0.993587
A8DZ36_SYNE7,1.0,0.999016,1.000170,1.006865,1.009448,1.011028,1.007514,1.007451,1.006162,1.002203,...,1.014753,1.012075,1.019956,1.014775,1.017129,1.012503,1.018721,1.012883,1.012384,1.010840
AAR_SYNE7,1.0,0.996677,1.000270,1.001493,1.001717,0.999874,1.005113,1.003955,1.005218,1.007475,...,0.986939,0.991755,0.991726,0.990433,0.999152,1.000068,1.001665,1.010297,1.011198,1.012243
ACCA_SYNE7,1.0,0.999332,1.002418,0.998179,0.996960,0.997356,0.999734,0.999929,0.997342,0.994947,...,0.996627,0.997264,0.995534,0.997335,1.001035,0.998438,0.996576,0.996670,0.992931,0.998058
ACCD_SYNE7,1.0,1.000217,1.001894,0.996700,0.995801,0.996660,0.995315,0.992459,0.995546,0.996920,...,0.998423,0.996643,0.995317,0.997427,0.996609,0.998488,0.999094,0.998946,0.999972,0.999765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YCF3_SYNE7,1.0,0.999054,0.998921,0.997062,0.995903,0.998899,1.001004,1.000349,1.000195,1.011672,...,0.996840,0.999791,0.987689,0.992117,0.994864,0.991431,0.998958,1.009900,1.004079,1.004495
YCF4_SYNE7,1.0,0.998014,1.002904,0.999648,0.992551,0.999662,0.997973,0.995586,0.991917,0.991401,...,0.990241,0.989643,0.985301,0.986501,0.984172,0.984373,0.983449,0.983721,0.984065,0.987650
YCF48_SYNE7,1.0,0.995618,1.001288,0.993663,0.988710,0.990204,0.990055,0.987671,0.986898,0.985461,...,0.992204,0.987702,0.988495,0.986933,0.986915,0.987908,0.986319,0.985916,0.986502,0.989541


Protein
6PGD_SYNE7      1.0
A8DZ36_SYNE7    1.0
AAR_SYNE7       1.0
ACCA_SYNE7      1.0
ACCD_SYNE7      1.0
               ... 
YCF3_SYNE7      1.0
YCF4_SYNE7      1.0
YCF48_SYNE7     1.0
YQGF_SYNE7      1.0
ZDS_SYNE7       1.0
Name: L_T0_A, Length: 1807, dtype: float64

## 5) Drop columns the ensure all the same number of columns

In [16]:
norm_metab_df = norm_metab_df.drop(columns=[col for col in norm_metab_df.columns if col not in norm_protein_df.columns])
norm_enzyme_activity_df = norm_enzyme_activity_df.drop(columns=[col for col in norm_enzyme_activity_df.columns if col not in norm_protein_df.columns])
clean_enzyme_activity_df = clean_enzyme_activity_df.drop(columns=[col for col in clean_enzyme_activity_df.columns if col not in norm_protein_df.columns])


## 6) Export and convert protein and reaction names

In [17]:
# export reaction ids to be comverted to metacyc names through metanex id mapper (web platform)
import csv

with open('../../data/circadian_experiments/processed_data/circadian_reaction_names.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for s in norm_enzyme_activity_df.index.tolist():
        writer.writerow([s])

In [18]:
# compare reaction names to metacyc reactions
metanetx_metacyc_mapper_df = pd.read_json('../../data/circadian_experiments/processed_data/circadian_metanetx_id_mapper.json').T
display(metanetx_metacyc_mapper_df)

,reference,xrefs,name,mnx_id
UPPDC1,rheaR:19865,"[bigg.reaction:R_UPPDC1, bigg.reaction:R_UPPDC...",4 MNXM13@MNXD1 + 1 MNXM735625@MNXD1 = 4 MNXM1@...,MNXR105140
GCALDDy,biggR:GCALDDy,"[bigg.reaction:GCALDDy, bigg.reaction:R_GCALDD...",2 MNXM1@MNXD1 + 1 MNXM222@MNXD1 + 1 MNXM738702...,MNXR130120
FUM,rheaR:12460,"[bigg.reaction:FUM, bigg.reaction:FUMg, bigg.r...",1 MNXM1107192@MNXD1 = 1 MNXM737340@MNXD1 + 1 W...,MNXR99705
PSIum,NaN,NaN,NaN,NaN
VPAMTr,rheaR:22912,"[bigg.reaction:R_VPAMTr, bigg.reaction:VPAMTr,...",1 MNXM1105732@MNXD1 + 1 MNXM732866@MNXD1 = 1 M...,MNXR105217
...,...,...,...,...
ANHMK,biggR:ANHMK,"[bigg.reaction:ANHMK, bigg.reaction:R_ANHMK, b...",1 MNXM3@MNXD1 + 1 MNXM90585@MNXD1 + 1 WATER@MN...,MNXR152984
PCXHtpp,mnx:MNXR03,"[MNXR03, bigg.reaction:HMR_1095, bigg.reaction...",1 MNXM01@MNXD1 = 1 MNXM01@MNXD2,MNXR03
Ktu,rheaR:29463,"[bigg.reaction:DM_k_c, bigg.reaction:DM_k_g, b...",1 MNXM95@MNXD1 = 1 MNXM95@MNXD2,MNXR100950
PCHLDA430,biggR:PCHLDA430,"[bigg.reaction:PCHLDA430, bigg.reaction:R_PCHL...",1 MNXM1093824@MNXD1 = 1 MNXM1093983@MNXD2 + 1 ...,MNXR160370


In [19]:
metanetx_metacyc_mapper_dict={}
for entry,row in metanetx_metacyc_mapper_df. iterrows():
    try:
        metacyc_name=''
        for id in row['xrefs']:
            if 'metacyc.reaction:' in id:
                metacyc_name = id.split(':')[1]
                break

        if metacyc_name!='':
            metanetx_metacyc_mapper_dict[entry] = metacyc_name

    except:
        print('skip')

display(metanetx_metacyc_mapper_dict)

skip
skip
skip
skip
skip
skip
skip
skip
skip
skip


{'UPPDC1': 'UROGENDECARBOX-RXN',
 'FUM': 'FUMHYDR-RXN',
 'VPAMTr': 'VALINE-PYRUVATE-AMINOTRANSFER-RXN',
 'MTAP': '5-METHYLTHIOADENOSINE-PHOSPHORYLASE-RXN',
 'G1PACT': '2.3.1.157-RXN',
 'CYTK1': 'RXN-11832',
 'NDPK7': 'DCDPKIN-RXN',
 'DTMPK': 'DTMPKI-RXN',
 'HSTPT': 'HISTAMINOTRANS-RXN',
 'GLYOX': 'GLYOXII-RXN',
 'SBP': 'SEDOHEPTULOSE-BISPHOSPHATASE-RXN',
 'DAPDC': 'DIAMINOPIMDECARB-RXN',
 'GND': 'RXN-9952',
 'PGK': 'PHOSGLYPHOS-RXN',
 'H4THDPS': 'DIHYDRODIPICSYN-RXN',
 'PRAGSr': 'GLYRIBONUCSYN-RXN',
 'GLUSfx': 'GLUTAMATE-SYNTHASE-FERREDOXIN-RXN',
 'GCATENEC': 'RXN1F-151',
 'SQD1': 'RXN-1223',
 'UDPG4E': 'UDPGLUCEPIM-RXN',
 'OHPBAT': 'PSERTRANSAMPYR-RXN',
 'ICHORS': 'ISOCHORSYN-RXN',
 'SHKK': 'SHIKIMATE-KINASE-RXN',
 'SPS': 'SUCROSE-PHOSPHATE-SYNTHASE-RXN',
 'ZNabcpp': 'ABC-63-RXN',
 'CMPA_1': 'N-CARBAMOYLPUTRESCINE-AMIDASE-RXN',
 'IGPS': 'IGPSYN-RXN',
 'OMPDC': 'OROTPDECARB-RXN',
 'GLYDHDA': 'GLYCINE-DEHYDROGENASE-RXN',
 'IMPD': 'IMP-DEHYDROG-RXN',
 'ORNDC': 'ORNDECARBOX-RXN',
 'OXADC'

In [20]:
# compare protein names with uniprot names
uniprot_df = pd.read_csv('../../data/circadian_experiments/processed_data/uniprotkb_taxonomy_id_1140_2024_06_18.tsv', delimiter='\t')

uniprot_df = uniprot_df.dropna(subset=['Gene Names (ordered locus)'])

uniprot_metacyc_mapper_dict={}

for entry,row in uniprot_df. iterrows():

    uniprot_metacyc_mapper_dict[row['Entry Name']] = row['Gene Names (ordered locus)']

display(uniprot_metacyc_mapper_dict)

{'CYNT_SYNE7': 'Synpcc7942_1447',
 'CCMN_SYNE7': 'Synpcc7942_1424',
 'CCMK2_SYNE7': 'Synpcc7942_1421',
 'CCMM_SYNE7': 'Synpcc7942_1423',
 'SASA_SYNE7': 'Synpcc7942_2114',
 'KATG_SYNE7': 'Synpcc7942_1656',
 'RBCX_SYNE7': 'Synpcc7942_1535',
 'RBL_SYNE7': 'Synpcc7942_1426',
 'RAF1_SYNE7': 'Synpcc7942_0833',
 'FBSB_SYNE7': 'Synpcc7942_0505',
 'CCMK4_SYNE7': 'Synpcc7942_0285',
 'RPAA_SYNE7': 'Synpcc7942_0095',
 'F16PA_SYNE7': 'Synpcc7942_2335',
 'KAIC_SYNE7': 'Synpcc7942_1216',
 'KAIB_SYNE7': 'Synpcc7942_1217',
 'KAIA_SYNE7': 'Synpcc7942_1218',
 'MCDA_SYNE7': 'Synpcc7942_1833',
 'MCDB_SYNE7': 'Synpcc7942_1834',
 'CYD_SYNE7': 'Synpcc7942_B2661',
 'CIKA_SYNE7': 'Synpcc7942_0644',
 'MURQ_SYNE7': 'Synpcc7942_2577',
 'PSBA1_SYNE7': 'Synpcc7942_0424',
 'PSBA2_SYNE7': 'Synpcc7942_1389; Synpcc7942_0893',
 'PHCB_SYNE7': 'Synpcc7942_1047; Synpcc7942_1052',
 'GLNB_SYNE7': 'Synpcc7942_0321',
 'PHCA_SYNE7': 'Synpcc7942_1048; Synpcc7942_1053',
 'CYSA_SYNE7': 'Synpcc7942_1680',
 'ISIA_SYNE7': 'Synpcc7942_

In [21]:
display(norm_metab_df)

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
sucr_e,1.0,1.015528,1.057453,1.368012,1.259317,1.600932,1.287267,1.562112,1.496894,2.812112,...,1.223602,3.369565,3.464286,3.524845,3.732919,4.993789,4.810559,6.880435,4.818323,6.503106


In [22]:
display(norm_enzyme_activity_df)

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
DTMPK,1.0,0.878411,1.053368,0.901785,0.692370,0.642508,0.745883,0.785337,0.653875,0.952481,...,1.166691,1.042845,0.840337,0.774831,0.967250,0.655043,0.776142,0.817852,1.097567,0.836904
ACOATA,1.0,1.119173,0.907860,1.442772,1.572014,1.481734,1.400597,1.337923,1.345316,1.069715,...,0.725190,1.407921,1.455456,1.425789,1.439274,1.297523,1.333004,1.441362,1.317623,1.243471
3HAD160,1.0,0.708981,0.713172,0.761793,0.482509,0.410421,0.686440,0.546205,0.434207,0.740276,...,1.160545,1.095287,0.930996,0.343369,0.701411,0.658629,0.642624,1.113902,0.801543,0.974567
RNDR3,1.0,0.972232,1.035877,1.967456,2.381647,2.409442,1.640272,1.840999,1.804969,2.015538,...,0.629824,2.006379,1.766751,2.449206,2.317056,2.716618,2.884303,4.039371,1.895945,1.721257
G5SD,1.0,0.994889,0.995240,1.036676,1.016329,0.956100,1.059373,1.116691,1.037369,0.926408,...,1.096962,1.132583,1.196123,1.063406,1.179780,1.167345,1.158137,1.005271,1.029116,0.949982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MDDCP5pp,1.0,1.058706,1.026764,1.063826,1.161266,1.435998,1.183432,1.316450,1.334434,1.182762,...,1.288010,1.050279,1.047362,1.197624,0.944384,0.966156,0.977049,0.859359,0.833692,0.956022
MDDEP2pp,1.0,1.056451,1.015238,0.919512,1.015814,1.164740,1.277589,1.331971,1.201462,0.948674,...,1.103004,1.112459,1.114460,1.027639,0.752190,0.870317,0.869828,0.633517,0.561680,0.610697
MDDEP3pp,1.0,1.056451,1.015238,0.919512,1.015814,1.164740,1.277589,1.331971,1.201462,0.948674,...,1.103004,1.112459,1.114460,1.027639,0.752190,0.870317,0.869828,0.633517,0.561680,0.610697


In [33]:
norm_enzyme_activity_mapped_df = norm_enzyme_activity_df.copy()
new_indexes=[]
count_found=0
for idn in norm_enzyme_activity_df.index:

    if idn in metanetx_metacyc_mapper_dict.keys():
        
        new_indexes.append(metanetx_metacyc_mapper_dict[idn])
        count_found+=1

    else:
        new_indexes.append(idn)
        print(idn +" : not in metanetx-metacyc mapping")
        
print(new_indexes)
print('Mapped '+str(count_found)+' of '+str(len(norm_enzyme_activity_df.index)))
norm_enzyme_activity_mapped_df.index = new_indexes



ACOATA : not in metanetx-metacyc mapping
3HAD160 : not in metanetx-metacyc mapping
RNDR3 : not in metanetx-metacyc mapping
HISTDa : not in metanetx-metacyc mapping
TRPS3 : not in metanetx-metacyc mapping
DMBZIDS2 : not in metanetx-metacyc mapping
ZCAROTDH2 : not in metanetx-metacyc mapping
U23GAAT2 : not in metanetx-metacyc mapping
LPOR : not in metanetx-metacyc mapping
GCALDDy : not in metanetx-metacyc mapping
MSBENZMT : not in metanetx-metacyc mapping
DESAT18a : not in metanetx-metacyc mapping
PHYFXOR : not in metanetx-metacyc mapping
TMPPP_1 : not in metanetx-metacyc mapping
RNDR1 : not in metanetx-metacyc mapping
TALA : not in metanetx-metacyc mapping
GLYCL : not in metanetx-metacyc mapping
ACONT : not in metanetx-metacyc mapping
HEMEAS_1 : not in metanetx-metacyc mapping
DHORD3um : not in metanetx-metacyc mapping
MTHFC : not in metanetx-metacyc mapping
HBZNPT : not in metanetx-metacyc mapping
COCHL_1 : not in metanetx-metacyc mapping
HSDy : not in metanetx-metacyc mapping
TRPS1 : 

In [ ]:
display(norm_enzyme_activity_mapped_df)

In [23]:
display(clean_enzyme_activity_df)

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
DTMPK,178.272000,156.596015,187.786003,160.762936,123.430272,114.541199,132.970048,140.003522,116.567522,169.800665,...,207.988251,185.909998,149.808645,138.130717,172.433615,116.775753,138.364354,145.800066,195.665426,149.196552
ACOATA,404.617348,452.836930,367.335777,583.770413,636.064063,599.535465,566.705682,541.346951,544.338243,432.825224,...,293.424502,569.669169,588.902948,576.898876,582.355346,525.000234,539.356590,583.200265,533.133324,503.129979
3HAD160,241.368269,171.125542,172.137169,183.872609,116.462434,99.062659,165.684901,131.836650,104.803827,178.679131,...,280.118856,264.367429,224.712967,82.878430,169.298459,158.972033,155.109085,268.860673,193.466938,235.229601
RNDR3,2704.125840,2629.037221,2801.141205,5320.248426,6440.273066,6515.433605,4435.500897,4978.291895,4880.863935,5450.268400,...,1703.122647,5425.501875,4777.518212,6622.961601,6265.610544,7346.078037,7799.519310,10922.966420,5126.873860,4654.496820
G5SD,1862.842245,1853.321909,1853.974964,1931.164774,1893.261108,1781.064048,1973.444367,2080.218994,1932.454235,1725.751855,...,2043.467058,2109.822641,2228.188346,1980.956985,2197.744805,2174.580408,2157.426358,1872.661400,1917.081478,1769.667134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MDDCP5pp,862.315684,912.938624,885.394530,917.353506,1001.377852,1238.283232,1020.492355,1135.195222,1150.703241,1019.913798,...,1110.671266,905.671643,903.156714,1032.730241,814.356937,833.131212,842.524346,741.038868,718.905554,824.392628
MDDEP2pp,2088.186065,2206.066541,2120.005135,1920.112322,2121.208949,2432.194648,2667.843509,2781.403299,2508.875283,1981.007757,...,2303.277298,2323.022180,2327.199807,2145.901311,1570.713476,1817.383987,1816.362633,1322.901518,1172.893312,1275.249363
MDDEP3pp,2088.186065,2206.066541,2120.005135,1920.112322,2121.208949,2432.194648,2667.843509,2781.403299,2508.875283,1981.007757,...,2303.277298,2323.022180,2327.199807,2145.901311,1570.713476,1817.383987,1816.362633,1322.901518,1172.893312,1275.249363


In [34]:
clean_enzyme_activity_mapped_df = clean_enzyme_activity_df.copy()
new_indexes=[]
count_found=0
for idn in clean_enzyme_activity_df.index:

    if idn in metanetx_metacyc_mapper_dict.keys():
        
        new_indexes.append(metanetx_metacyc_mapper_dict[idn])
        count_found+=1

    else:
        new_indexes.append(idn)
        print(idn +" : not in metanetx-metacyc mapping")
        
print(new_indexes)
print('Mapped '+str(count_found)+' of '+str(len(clean_enzyme_activity_df.index)))
clean_enzyme_activity_mapped_df.index = new_indexes

ACOATA : not in metanetx-metacyc mapping
3HAD160 : not in metanetx-metacyc mapping
RNDR3 : not in metanetx-metacyc mapping
HISTDa : not in metanetx-metacyc mapping
TRPS3 : not in metanetx-metacyc mapping
DMBZIDS2 : not in metanetx-metacyc mapping
ZCAROTDH2 : not in metanetx-metacyc mapping
U23GAAT2 : not in metanetx-metacyc mapping
LPOR : not in metanetx-metacyc mapping
GCALDDy : not in metanetx-metacyc mapping
MSBENZMT : not in metanetx-metacyc mapping
DESAT18a : not in metanetx-metacyc mapping
PHYFXOR : not in metanetx-metacyc mapping
TMPPP_1 : not in metanetx-metacyc mapping
RNDR1 : not in metanetx-metacyc mapping
TALA : not in metanetx-metacyc mapping
GLYCL : not in metanetx-metacyc mapping
ACONT : not in metanetx-metacyc mapping
HEMEAS_1 : not in metanetx-metacyc mapping
DHORD3um : not in metanetx-metacyc mapping
MTHFC : not in metanetx-metacyc mapping
HBZNPT : not in metanetx-metacyc mapping
COCHL_1 : not in metanetx-metacyc mapping
HSDy : not in metanetx-metacyc mapping
TRPS1 : 

In [ ]:
# list ones that cannot map and put as issue on github for s elongatus 

In [24]:
display(norm_protein_df)

,L_T0_A,L_T0_B,L_T0_C,L_T2_A,L_T2_B,L_T2_C,L_T4_A,L_T4_B,L_T4_C,L_T8_A,...,D_T0_C,D_T2_A,D_T2_B,D_T2_C,D_T4_A,D_T4_B,D_T4_C,D_T8_A,D_T8_B,D_T8_C
Protein,,,,,,,,,,,,,,,,,,,,,
6PGD_SYNE7,1.0,0.996753,0.999996,0.992003,0.990776,0.992409,0.989355,0.986516,0.986929,0.985514,...,1.005694,1.004651,1.001349,1.004406,1.002110,1.000040,1.000436,0.991281,0.992127,0.993587
A8DZ36_SYNE7,1.0,0.999016,1.000170,1.006865,1.009448,1.011028,1.007514,1.007451,1.006162,1.002203,...,1.014753,1.012075,1.019956,1.014775,1.017129,1.012503,1.018721,1.012883,1.012384,1.010840
AAR_SYNE7,1.0,0.996677,1.000270,1.001493,1.001717,0.999874,1.005113,1.003955,1.005218,1.007475,...,0.986939,0.991755,0.991726,0.990433,0.999152,1.000068,1.001665,1.010297,1.011198,1.012243
ACCA_SYNE7,1.0,0.999332,1.002418,0.998179,0.996960,0.997356,0.999734,0.999929,0.997342,0.994947,...,0.996627,0.997264,0.995534,0.997335,1.001035,0.998438,0.996576,0.996670,0.992931,0.998058
ACCD_SYNE7,1.0,1.000217,1.001894,0.996700,0.995801,0.996660,0.995315,0.992459,0.995546,0.996920,...,0.998423,0.996643,0.995317,0.997427,0.996609,0.998488,0.999094,0.998946,0.999972,0.999765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YCF3_SYNE7,1.0,0.999054,0.998921,0.997062,0.995903,0.998899,1.001004,1.000349,1.000195,1.011672,...,0.996840,0.999791,0.987689,0.992117,0.994864,0.991431,0.998958,1.009900,1.004079,1.004495
YCF4_SYNE7,1.0,0.998014,1.002904,0.999648,0.992551,0.999662,0.997973,0.995586,0.991917,0.991401,...,0.990241,0.989643,0.985301,0.986501,0.984172,0.984373,0.983449,0.983721,0.984065,0.987650
YCF48_SYNE7,1.0,0.995618,1.001288,0.993663,0.988710,0.990204,0.990055,0.987671,0.986898,0.985461,...,0.992204,0.987702,0.988495,0.986933,0.986915,0.987908,0.986319,0.985916,0.986502,0.989541


In [35]:
norm_protein_mapped_df = norm_protein_df.copy()
new_indexes=[]
count_found=0
for idn in norm_protein_df.index:

    if idn in uniprot_metacyc_mapper_dict.keys():
        
        new_indexes.append(uniprot_metacyc_mapper_dict[idn])
        count_found+=1

    else:
        new_indexes.append(idn)
        print(idn +" : not in uniprot-metacyc mapping")
        
print(new_indexes)
print('Mapped '+str(count_found)+' of '+str(len(norm_protein_df.index)))
norm_protein_mapped_df.index = new_indexes

A8DZ36_SYNE7 : not in uniprot-metacyc mapping
O05160_SYNE7 : not in uniprot-metacyc mapping
O05163_SYNE7 : not in uniprot-metacyc mapping
O06867_SYNE7 : not in uniprot-metacyc mapping
O07334_SYNE7 : not in uniprot-metacyc mapping
O32384_SYNE7 : not in uniprot-metacyc mapping
O50211_SYNE7 : not in uniprot-metacyc mapping
O50544_SYNE7 : not in uniprot-metacyc mapping
O54307_SYNE7 : not in uniprot-metacyc mapping
O68868_SYNE7 : not in uniprot-metacyc mapping
P72543_SYNE7 : not in uniprot-metacyc mapping
P72561_SYNE7 : not in uniprot-metacyc mapping
PCXA_SYNE7 : not in uniprot-metacyc mapping
PHCA1_SYNP6 : not in uniprot-metacyc mapping
PHCB1_SYNP6 : not in uniprot-metacyc mapping
Q31LZ8_SYNE7 : not in uniprot-metacyc mapping
Q31S42_SYNE7 : not in uniprot-metacyc mapping
Q53572_SYNE7 : not in uniprot-metacyc mapping
Q55019_SYNE7 : not in uniprot-metacyc mapping
Q769G0_SYNE7 : not in uniprot-metacyc mapping
Q8GJM5_SYNE7 : not in uniprot-metacyc mapping
Q8GJM6_SYNE7 : not in uniprot-metacyc 

## 7) Save to processed data

In [37]:
norm_metab_df_fname = '../../data/circadian_experiments/processed_data/normalized_sucrose_metabolomics.csv'
norm_enzyme_activity_df_fname = '../../data/circadian_experiments/processed_data/normalized_enzyme_activity.csv'
enzyme_activity_df_fname = '../../data/circadian_experiments/processed_data/cleaned_enzyme_activity.csv'
norm_protein_df_fname = '../../data/circadian_experiments/processed_data/normalized_protein_abundance.csv'

norm_enzyme_activity_mapped_df_fname = '../../data/circadian_experiments/processed_data/normalized_enzyme_activity_mapped.csv'
enzyme_activity_mapped_df_fname = '../../data/circadian_experiments/processed_data/cleaned_enzyme_activity_mapped.csv'
norm_protein_mapped_df_fname = '../../data/circadian_experiments/processed_data/normalized_protein_abundance_mapped.csv'

norm_metab_df.to_csv(norm_metab_df_fname)

norm_enzyme_activity_df.to_csv(norm_enzyme_activity_df_fname)

clean_enzyme_activity_df.to_csv(enzyme_activity_df_fname)

norm_protein_df.to_csv(norm_protein_df_fname)


norm_enzyme_activity_mapped_df.to_csv(norm_enzyme_activity_mapped_df_fname)

clean_enzyme_activity_mapped_df.to_csv(enzyme_activity_mapped_df_fname)

norm_protein_mapped_df.to_csv(norm_protein_mapped_df_fname)
